<h3> Data Cleaning </h3>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [77]:
import numpy as np
import pandas as pd
from rapidfuzz import process
from os.path import exists

In [3]:
#function that makes dictionary that combines 3 full name columns into 1
def genFullNameDict(df):    
    namemerge_pre = dict()
    for x in df.index:
        marker = False
        row = df.loc[x][['full name 1', 'full name 2', 'full name 3']]
        #identify number of unique names
        if len(row[row.notna()].unique()) > 1:
            row_names = row[row.notna()].unique()
            for name in row_names:
                if name in namemerge_pre.keys() and not marker:
                    #add to dictionary if entry already exists
                    namemerge_pre[name].extend([n for n in row_names if n != name and n not in namemerge_pre[name]])
                    marker = True
            #add to dictionary using shortest name as key
            if not marker:
                minname = row_names[0]
                for name in row_names:
                    if len(name) < len(minname):
                        minname = name
                namemerge_pre[minname] = [n for n in row_names if n != minname]
    
    #invert the dictionary - swap keys and values
    namemerge = dict()
    for key in namemerge_pre.keys():
        vals = namemerge_pre[key]
        #iterate through list of values
        for val in vals:
            namemerge[val] = key
    
    return namemerge

In [4]:
#use the fullname 1/2/3 columns to generate full name column
def setFullName(df):
    for x in df.index:
        marker = False
        row = df.loc[x][['full name 1', 'full name 2', 'full name 3']]
        if len(row[row.notna()].unique()) == 0:
            df.loc[x, 'full name'] = np.nan
        else:
            df.loc[x, 'full name'] = row[row.notna()].unique()[0]
    return df

In [5]:
def genFuzzyDict(df):
    namelst = df['full name'].drop_duplicates()
    fn_fuzzy_pre = dict()
    for name in namelst:
        marker = False
        if not pd.isnull(name):
            #find matches for name
            match = process.extract(name, [x for x in namelst if x != name and not pd.isnull(x)], limit = 1)[0]
            if match[1]>90:
                #add suitable matches to dictionary
                for nm in [match[0], name]:
                    if nm in fn_fuzzy_pre.keys() and not marker:
                        fn_fuzzy_pre[nm].extend([n for n in [match[0], name] if n != nm and n not in fn_fuzzy_pre[nm]])
                        marker = True
                if not marker:
                    if len(name) < len(match[0]):
                        fn_fuzzy_pre[name] = [match[0]]
                    else:
                        fn_fuzzy_pre[match[0]] = [name]
    #invert dictionary
    fn_fuzzy = dict()
    for key in fn_fuzzy_pre.keys():
        vals = fn_fuzzy_pre[key]
        for val in vals:
            fn_fuzzy[val] = key
    
    return fn_fuzzy

In [6]:
def transformdf(df, state):
    df['full name 1'] = df['First Name'] + " " + df['Last Name']
    df['full name 2'] = df['First Name.1'] + " " + df['Last Name.1']
    df['full name 3'] = df['First Name.2'].apply(lambda x: x if type(x) == str else "") + " " + df['Last Name.2']
    df['state'] = state
    
    namemerge = genFullNameDict(df)
    for col in ['full name 1', 'full name 2', 'full name 3']:    
        df[col] = df[col].apply(lambda x: namemerge[x] if x in namemerge.keys() else x)

    df['full name'] = np.nan
    df = setFullName(df)

    fn_fuzzy = genFuzzyDict(df)
    df['full name'] = df['full name'].apply(lambda x: x if x not in fn_fuzzy.keys() else fn_fuzzy[x])
    
    df_agg = df[['full name', 'state', '6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']]
    return df_agg

In [7]:
def transformonecoldf(df, state):
    df['full name'] =  df['First Name'] + " " + df['Last Name']
    df['state'] = state
    fn_fuzzy = genFuzzyDict(df)
    df['full name'] = df['full name'].apply(lambda x: x if x not in fn_fuzzy.keys() else fn_fuzzy[x])    
    df_agg = df[['full name', 'state', '6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']]
    return df_agg

# Connecticut Continental Debt Dataset Matching

In [8]:
#prepare loan dataset
CT_CD = pd.read_excel("Data/Post1790/CT/CT_post1790_CD_ledger.xlsx", header = 13, usecols = 'H, I, N, O, X, Y, AD, AE, AN, AO, AT, AU')
CT_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
CT_CD_agg = transformdf(CT_CD, 'CT')

# Maryland Continental Debt Dataset Matching

In [9]:
#prepare loan dataset
MD_CD = pd.read_excel("Data/Post1790/MD/MD_post1790_CD.xlsx", header = 11, usecols = 'G, H, L, M, U, V, Z, AA, AI, AJ, AN, AO')
MD_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
MD_CD_agg = transformdf(MD_CD, 'MD')

In [10]:
cumulative_CD = pd.concat([CT_CD_agg, MD_CD_agg])

# North Carolina Continental Debt Dataset Matching

In [11]:
#prepare loan dataset
NC_CD = pd.read_excel("Data/Post1790/NC/T695_R4_NC_CD.xlsx", header = 11, usecols = 'J, K, W, X, Z, AA, AC, AD ')
NC_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']
NC_CD_agg = transformonecoldf(NC_CD, 'NC')

In [12]:
cumulative_CD = pd.concat([NC_CD_agg, cumulative_CD])

# New Hampshire Continental Debt Dataset Matching

In [13]:
#prepare loan dataset
NH_CD = pd.read_excel("Data/Post1790/NH/T652_R6_New_Hampshire_CD.xlsx", header = 10, usecols = 'I, J, N, O, P, Q, R, S')
NH_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents',  '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']
NH_CD_agg = transformonecoldf(NH_CD, 'NH')

In [14]:
cumulative_CD = pd.concat([NH_CD_agg, cumulative_CD])

# New York Continental Debt Dataset Matching

In [15]:
#prepare loan dataset
NY_CD = pd.read_excel("Data/Post1790/NY/NY_1790_CD.xlsx", header = 11, usecols = 'H, I, M, N, X, Y, AC, AD, AM, AN, AR, AS')
NY_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
NY_CD_agg = transformdf(NY_CD, 'NY')

In [16]:
cumulative_CD = pd.concat([NY_CD_agg, cumulative_CD])

# South Carolina Continental Debt Dataset Matching

In [17]:
#prepare loan dataset
SC_CD = pd.read_excel("Data/Post1790/SC/Post_1790_South_Carolina_CD.xlsx", header = 11, usecols = 'D, E, M, N, S, T, AB, AC, AH, AI, AQ, AR')
SC_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
SC_CD_agg = transformdf(SC_CD, 'SC')

In [18]:
cumulative_CD = pd.concat([SC_CD_agg, cumulative_CD])

# Pennsylvania Continental Debt Dataset Matching

In [19]:
#prepare loan dataset
PA_CD = pd.read_excel("Data/Post1790/PA/PA_post1790_CD.xlsx", header = 11, usecols = 'G, H, L, M, U, V, Z, AA, AI, AJ, AO, AP')
PA_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
PA_CD_agg = transformdf(PA_CD, 'PA')

In [20]:
cumulative_CD = pd.concat([PA_CD_agg, cumulative_CD])

# Rhode Island Continental Debt Dataset Matching

In [21]:
#prepare loan dataset
RI_CD = pd.read_excel("Data/Post1790/RI/T653_Rhode_Island_CD.xlsx", header = 11, usecols = 'G, H, L, M, U, V, Z, AA, AI, AJ, AN, AO')
RI_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
RI_CD_agg = transformdf(RI_CD, 'RI')

In [22]:
cumulative_CD = pd.concat([RI_CD_agg, cumulative_CD])

# Virginia Continental Debt Dataset Matching

In [23]:
#prepare loan dataset
VA_CD = pd.read_excel("Data/Post1790/VA/VA_CD.xlsx", header = 11, usecols = 'H, I, K, L, U, V, X, Y, AH, AI, AK, AL')
VA_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents', 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
VA_CD_agg = transformdf(VA_CD, 'VA')

In [24]:
cumulative_CD = pd.concat([VA_CD_agg, cumulative_CD])

# Georgia Continental Debt Dataset Matching

In [25]:
#prepare loan dataset
GA_CD = pd.read_excel("Data/Post1790/GA/T694_GA_Loan_Office_CD.xlsx", header = 10, usecols = 'Q, R, Z, AA, AB, AC, AD, AE')
GA_CD.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents',  '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']
GA_CD_agg = transformonecoldf(GA_CD, 'GA')

In [26]:
cumulative_CD = pd.concat([GA_CD_agg, cumulative_CD])

# New Jersey Continental Debt Dataset Matching

In [27]:
#prepare loan dataset
NJ_CD = pd.read_excel("Data/Post1790/NJ/NJ_3_percent_stock_T698_R1_R2.xlsx", header = 11, usecols = 'D, E, L, M')
NJ_CD.columns = ['First Name', 'Last Name', '3p_Dollar', '3p_Cents']
NJ_CD['full name'] =  NJ_CD['First Name'] + " " + NJ_CD['Last Name']
NJ_CD['state'] = 'NJ'
fn_fuzzy = genFuzzyDict(NJ_CD)
NJ_CD['full name'] = NJ_CD['full name'].apply(lambda x: x if x not in fn_fuzzy.keys() else fn_fuzzy[x])    
NJ_CD_agg = NJ_CD[['full name', 'state', '3p_Dollar', '3p_Cents']]

# Summary Analysis

In [28]:
cumulative_CD = pd.concat([NJ_CD_agg, cumulative_CD])

In [29]:
cumulative_CD.reset_index(drop = True, inplace = True)

## How many unique individuals were issued 6 percent stocks or deferred 6 percent stocks in 1790 and after?

In [170]:
stocks_6 = cumulative_CD[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents']].dropna(thresh = 1).index
print('table of number of unique individuals issued 6% stocks (normal or deferred) by state')
cumulative_CD[cumulative_CD['full name'].apply(lambda x: not pd.isnull(x))].loc[stocks_6].groupby('state')['full name'].apply(lambda x: len(x.unique()))

table of number of unique individuals issued 6% stocks (normal or deferred) by state


state
CT    705
GA     54
MD    333
NC     53
NH    169
NY    158
PA    285
RI    409
SC    256
VA    421
Name: full name, dtype: int64

# How many of these individuals
- were original purchasers of loan office certicates of the same state as the 6 percent stock?
- were original purchasers of loan office certicates issued from another state?
- were original recipients of liquidated debtcerti cates issued by the same-state loan office? other state loan offices?
- were original recipients of the Pierce Certicates?

In [197]:
def fuzzy_merge(lst1, lst2, threshold=85, limit = 100):
    """
    :param df_1: the left list to join
    :param df_2: the right list to join
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    
    delegates = pd.Series([x for x in lst1.unique() if not pd.isnull(x)])
    possible =  [x for x in lst2.unique().tolist() if type(x) == str]

    
    #get matches
    #process.extract uses a combination of all four fuzzywuzzy scores
    matches = delegates.apply(lambda x: process.extract(x, possible, limit=limit, score_cutoff = threshold))
    
    match_df = pd.DataFrame(columns = ['Delegates', 'Loan Matches'])
    
    for delegate, matchset in zip(delegates, matches):
        matchset_thres = [name for name in matchset if name[1] >= threshold]
        if len(matchset_thres) == 0:
            add_df = pd.DataFrame(data = {'Delegates': [delegate], 'Loan Matches': [""], 'Scores': [0]})
            match_df = pd.concat([match_df, add_df])
        else:
            delegate_lst = [delegate] * len(matchset_thres)
            add_df = pd.DataFrame(data = {'Delegates': delegate_lst, 
                                          'Loan Matches': [x[0] for x in matchset_thres],
                                          'Scores': [x[1] for x in matchset_thres]})
            match_df = pd.concat([match_df, add_df])

    return match_df

In [172]:
#function for performing the second step of the match
def matchFunction(lst1, lst2, score = 90):
    lst1 = list(set(lst1))
    lst2 = list(set(lst2))
    threshold = min(len(lst1), len(lst2))
    matches = 0
    for wd1 in lst1:
        for wd2 in lst2:
            if process.extract(wd1, [wd2])[0][1] > score:
                matches+=1
    return matches >= threshold

In [173]:
def produceMatches(delegates, debt, delegate_names, debt_names):
    initial = True
    join_df = pd.DataFrame()
    for del_name in delegate_names:
        for debt_name in debt_names:
            if initial:
                join_df = fuzzy_merge(delegates[del_name], debt[debt_name])
                initial = False
            else:
                add_df = fuzzy_merge(delegates[del_name], debt[debt_name])
                join_df = pd.concat([join_df, add_df])
    join_df = join_df.drop_duplicates().reset_index(drop = True)
    join_df = join_df[join_df['Scores'].apply(lambda x: x != 0)]
    join_df = join_df[join_df['Loan Matches'].apply(lambda x: not pd.isnull(x))]    
    
    join_df_p2 = join_df[join_df['Loan Matches'].apply(lambda x: len(list(set(x.replace("??", "").strip().split(" "))))>=2)]
    join_df_p2_final = join_df_p2[[matchFunction(x.split(" "), y.split(" ")) for x, y in zip(join_df_p2['Delegates'], join_df_p2['Loan Matches'])]]
    return join_df_p2_final

In [174]:
def stringConvert(x):
    return x if type(x) == str else ""

In [175]:
loan_office = pd.read_csv('Data/Pre1790/cleaned/loan_office_certificates_9_states_cleaned.csv', index_col = 0)

In [176]:
states = ['NH', 'MA', 'CT', 'NY', 'NJ', 'PA', 'DE', 'MD', 'VA']
num_names = [1, 2, 2, 3, 2, None, 2, None, None]
state_names = dict(zip(np.arange(1, 10, 1), states))
loan_office['State Name'] = loan_office['State'].apply(lambda x: state_names[x])
loan_office['Full Name 1'] = (loan_office['First Name 1 '].apply(lambda x: stringConvert(x)) + " " + loan_office['Last Name 1 '].apply(lambda x: stringConvert(x))).apply(lambda x: np.nan if x.strip() == "" else x)
loan_office['Full Name 2'] = (loan_office['First Name 2'].apply(lambda x: stringConvert(x)) + " " + loan_office['Last Name 2'].apply(lambda x: stringConvert(x))).apply(lambda x: np.nan if x.strip() == "" else x)
loan_office['Full Name 3'] = (loan_office['First Name 3'].apply(lambda x: stringConvert(x)) + " " + loan_office['Last Name 3'].apply(lambda x: stringConvert(x))).apply(lambda x: np.nan if x.strip() == "" else x)

### How many individuals were original purchasers of loan office certicates of the same state as the 6 percent stock?

In [177]:
def loanOfficeSameState(state):
    state_ind = cumulative_CD[cumulative_CD['state'] == state][['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents']].dropna(thresh = 1).index
    if len(state_ind) != 0:
        state_cd = cumulative_CD.loc[state_ind][['full name', 'state']].drop_duplicates()
        state_cd[state_cd['full name'].apply(lambda x: len(x.strip().split(" "))>1 if not pd.isnull(x) else False)]
        state_cd.columns = ['cd name 1', 'cd state']
        loan_office_state = loan_office[loan_office['State Name'] == state][['Full Name 1', 'Full Name 2', 'Full Name 3', 'State Name']].drop_duplicates()
        loan_office_state.columns = ['loan office name 1', 'loan office name 2', 'loan office name 3', 'loan office state']
        matches = produceMatches(state_cd, loan_office_state, delegate_names = ['cd name 1'], debt_names = ['loan office name 1', 'loan office name 2', 'loan office name 3'])
        return len(set(matches['Delegates']))

In [178]:
state_results = dict()

In [179]:
def storeStateResult(result, dictionary, key):
    dictionary[key] = result
    print(key, result)

### How many individuals were original purchasers of loan office certicates issued from another state?

In [182]:
def loanOfficeDifState(state):
    state_ind = cumulative_CD[cumulative_CD['state'] == state][['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents']].dropna(thresh = 1).index
    if len(state_ind) != 0:
        state_cd = cumulative_CD.loc[state_ind][['full name', 'state']].drop_duplicates()
        state_cd[state_cd['full name'].apply(lambda x: len(x.strip().split(" "))>1 if not pd.isnull(x) else False)]
        state_cd.columns = ['cd name 1', 'cd state']
        loan_office_nostate = loan_office[loan_office['State Name'] != state][['Full Name 1', 'Full Name 2', 'Full Name 3', 'State Name']].drop_duplicates()
        loan_office_nostate.columns = ['loan office name 1', 'loan office name 2', 'loan office name 3', 'loan office state']
        matches = produceMatches(state_cd, loan_office_nostate, delegate_names = ['cd name 1'], debt_names = ['loan office name 1', 'loan office name 2', 'loan office name 3'])
        return len(set(matches['Delegates']))

In [183]:
for state in states:
    print(state, loanOfficeDifState(state))

NH 44
MA None
CT 109
NY 27
NJ None
PA 26
DE None
MD 63
VA 90


In [184]:
def liquidatedStateDebt(state, file, num_names, df_ind = False):
    state_ind = cumulative_CD[cumulative_CD['state'] == state][['6p_Dollar', '6p_Cents']].dropna(thresh = 1).index
    if len(state_ind) != 0:
        state_cd = cumulative_CD.loc[state_ind][['full name', 'state']].drop_duplicates()
        state_cd[state_cd['full name'].apply(lambda x: len(x.strip().split(" "))>1 if not pd.isnull(x) else False)]
        state_cd.columns = ['cd name 1', 'cd state']
        datafile = 'Data/Pre1790/cleaned/'+file
        if exists(datafile):
            state_cert = pd.read_csv(datafile, index_col = 0)
            namelst = []
            state_cert['Full Name'] = state_cert['First name'] + " " + state_cert['Last name'] 
            namelst.append('Full Name')
            if num_names > 1:
                for i in np.arange(2, num_names+1, 1):
                    fullname_str = 'Full Name ' + str(i)
                    state_cert[fullname_str] = state_cert['First name ' + str(i)] + " " + state_cert['Last name ' + str(i)] 
                    namelst.append(fullname_str)
            state_cert_names = state_cert[namelst].drop_duplicates()
            matches = produceMatches(state_cd, state_cert_names, delegate_names = ['cd name 1'], debt_names = namelst)
            if df_ind:
                return matches
            return len(set(matches['Delegates']))

In [186]:
state_liquid_results = dict()

In [187]:
for state, num_name in zip(states, num_names):
    if state != "PA":
        file = 'liquidated_debt_certificates_'+state+'_cleaned.csv'
        result = liquidatedStateDebt(state, file, num_name)
    else:
        file1 = 'liquidated_debt_certificates_PA_story_cleaned.csv'
        df1 = liquidatedStateDebt('PA', file1, 1, df_ind = True)
        file2 = 'liquidated_debt_certificates_PA_stelle_cleaned.csv'
        df2 = liquidatedStateDebt('PA', file2, 2, df_ind = True)
        df = pd.concat([df1, df2])
        result = len(set(df['Delegates']))
        state_liquid_results[state] = result
    state_liquid_results[state] = result

['Full Name'] NH
['Full Name', 'Full Name 2'] CT
['Full Name', 'Full Name 2', 'Full Name 3'] NY
['Full Name'] PA
['Full Name', 'Full Name 2'] PA


In [188]:
state_liquid_results

{'NH': 28,
 'MA': None,
 'CT': 100,
 'NY': 38,
 'NJ': None,
 'PA': 23,
 'DE': None,
 'MD': None,
 'VA': None}